In [1]:
%config IPCompleter.greedy = True  
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

from seaborn import countplot
from matplotlib.pyplot import figure, show

from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.models import FactorRange, ColumnDataSource, TableColumn, DataTable, CustomJS
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler

from bokeh.models.widgets import Panel, Tabs, Select
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models.widgets import CheckboxGroup
from bokeh.layouts import column, row, WidgetBox  
from bokeh.core.properties import value 

In [2]:
host = "mysql.nm-interactive.net"
port = 3306
user = "windesheim"
password = "pMjq357Kdee7Sx8C"
database = "windesheim"
client = "mysql"

#Installeer eerst Mysqlcient in Anaconda Navigator
engine = create_engine("{0}://{1}:{2}@{3}:{4}/{5}".format(client, user, password, host, port, database))

datasource = pd.read_sql_query("SELECT * FROM source WHERE KWALIFICATIENIVEAU = 4",engine)
wervingsgebieden = pd.read_sql_query("SELECT * FROM wervingsgebieden",engine)
datasource.info()
wervingsgebieden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169415 entries, 0 to 169414
Data columns (total 14 columns):
nr                    169415 non-null int64
PC4_LEERL             169415 non-null int64
KWALIFICATIENIVEAU    169415 non-null int64
KWALIFICATIE CODE     169415 non-null int64
KWALIFICATIE NAAM     169415 non-null object
MBO SECTOR            169415 non-null object
DOMEIN                169415 non-null object
SECTORUNIT SBB        169415 non-null object
GESLACHT              169415 non-null object
2013   AANT           169415 non-null int64
2014   AANT           169415 non-null int64
2015   AANT           169415 non-null int64
2016   AANT           169415 non-null int64
2017   AANT           169415 non-null int64
dtypes: int64(9), object(5)
memory usage: 18.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 6 columns):
index            44 non-null int64
PostcodeStart    44 non-null int64
PostcodeEind     44 non-null int64
Gemeente  

In [3]:
# Laat n records zien van de datasource
datasource.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,2013 AANT,2014 AANT,2015 AANT,2016 AANT,2017 AANT
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,1,0,0,0,0
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,1,0,0,0,0
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,0,0,0,2,0
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,1,0,0,0,0
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,0,1,0,0,0


In [4]:
datasource.dtypes

nr                     int64
PC4_LEERL              int64
KWALIFICATIENIVEAU     int64
KWALIFICATIE CODE      int64
KWALIFICATIE NAAM     object
MBO SECTOR            object
DOMEIN                object
SECTORUNIT SBB        object
GESLACHT              object
2013   AANT            int64
2014   AANT            int64
2015   AANT            int64
2016   AANT            int64
2017   AANT            int64
dtype: object

In [5]:
# Laat n records zien van de ingevoerde wervingsgebieden
wervingsgebieden.head()

,index,PostcodeStart,PostcodeEind,Gemeente,Locatie,Categorie
0,0,7940,7999,Meppel,Zwolle,Thuismarkt
1,1,8000,8069,Zwolle,Zwolle,Thuismarkt
2,2,8070,8099,Nunspeet,Zwolle,Thuismarkt
3,3,8100,8159,Raalte,Zwolle,Thuismarkt
4,4,8160,8199,Epe,Zwolle,Thuismarkt


In [6]:
wervingsgebieden.dtypes

index             int64
PostcodeStart     int64
PostcodeEind      int64
Gemeente         object
Locatie          object
Categorie        object
dtype: object

In [7]:
# tijdelijke dataframe - 1000 rows
ds = datasource.sample(n=1000)


In [8]:
# Functie voor het checken van de locatie en categorie, stukje sneller gemaakt ook.
def check_locatie(PostCode):
    test = wervingsgebieden.loc[(wervingsgebieden.PostcodeStart <= PostCode) & (wervingsgebieden.PostcodeEind >= PostCode), ['Locatie', 'Categorie']]
    if test['Locatie'].empty:
        r = pd.Series(['',''])
    else:
        r = pd.Series([test['Locatie'].iloc[0], test['Categorie'].iloc[0]])
    return r

#voeg kolommen toe aan datasource 
datasource[['Locatie', 'Categorie']] = datasource.apply(lambda row: check_locatie(row["PC4_LEERL"]), axis=1)


In [9]:
#Welke soorten zitten er in Locatie?
datasource.Locatie.unique()

#Welke soorten zitten in Categorie?
datasource.Categorie.unique()

array(['Aangrenzend Primair', '', 'Thuismarkt', 'Aangrenzend Secundair'],
      dtype=object)

In [10]:
datasource.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,2013 AANT,2014 AANT,2015 AANT,2016 AANT,2017 AANT,Locatie,Categorie
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,1,0,0,0,0,Flevoland,Aangrenzend Primair
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,1,0,0,0,0,Flevoland,Aangrenzend Primair
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,0,0,0,2,0,Flevoland,Aangrenzend Primair
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,1,0,0,0,0,Flevoland,Aangrenzend Primair
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,0,1,0,0,0,Flevoland,Aangrenzend Primair


# 1. Data-Melter

Een punt van aandacht is het feit dat de data niet (mooi) allemaal in gelijke kolommen zitten. Zo zijn bijvoorbeeld de jaargegevens over diverse kolommen verdeeld. Om hiermee goed te kunnen werken, dient deze data onder elkaar te worden gezet.

In [11]:
# Originele dataset behouden, in kopie werken:
dsc = datasource.copy()

In [12]:
# Verkrijg alle 'Jaartal' kolommen (de kolommen die ge-melt dienen te worden)
jaarCols = [col for col in dsc.columns if 'AANT' in col]
jaarCols

['2013   AANT', '2014   AANT', '2015   AANT', '2016   AANT', '2017   AANT']

In [13]:
# Wat zijn dan de id vars? (kolommen die gelijk moeten blijven)
otherCols = [col for col in dsc.columns if not 'AANT' in col]
otherCols

['nr',
 'PC4_LEERL',
 'KWALIFICATIENIVEAU',
 'KWALIFICATIE CODE',
 'KWALIFICATIE NAAM',
 'MBO SECTOR',
 'DOMEIN',
 'SECTORUNIT SBB',
 'GESLACHT',
 'Locatie',
 'Categorie']

In [14]:
# Verander de jaartalkolomnamen naar pure jaartallen (gebruik alleen eerste 4 karakters van kolomnaam)
jaarCleanCols = [str(x)[:4] for x in jaarCols]
jaarCleanCols

['2013', '2014', '2015', '2016', '2017']

In [15]:
# Maak een dictionary voor de rename functie
jaarDict = dict(zip(jaarCols, jaarCleanCols))
jaarDict

{'2013   AANT': '2013',
 '2014   AANT': '2014',
 '2015   AANT': '2015',
 '2016   AANT': '2016',
 '2017   AANT': '2017'}

In [16]:
# Rename de betreffende colommen
dsc = dsc.rename(columns=jaarDict)

In [17]:
# Voer werkelijke melt uit
dscMelt = pd.melt(dsc, id_vars=otherCols, var_name="Jaar", value_name="Aantal")

In [18]:
dscMelt.head()

,nr,PC4_LEERL,KWALIFICATIENIVEAU,KWALIFICATIE CODE,KWALIFICATIE NAAM,MBO SECTOR,DOMEIN,SECTORUNIT SBB,GESLACHT,Locatie,Categorie,Jaar,Aantal
0,1424,1011,4,10388,Horeca-ondernemer/-manager,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,1
1,1425,1011,4,10801,Restauratiemedewerker,techniek,Bouw en infra,Sectorunit techniek en gebouwde omgeving,M,Flevoland,Aangrenzend Primair,2013,1
2,1426,1011,4,90038,Artiest (Muziek),techniek,Zorg en welzijn,Sectorunit creatieve industrie en ICT,M,Flevoland,Aangrenzend Primair,2013,0
3,1427,1011,4,90290,Ondernemer detailhandel,economie,Handel en ondernemerschap,Sectorunit handel,M,Flevoland,Aangrenzend Primair,2013,1
4,1428,1011,4,90301,Ondernemer horeca/bakkerij (Manager/ondernemer...,economie,Horeca en bakkerij,"Sectorunit voedsel, groen en gastvrijheid",M,Flevoland,Aangrenzend Primair,2013,0


In [19]:
dscMelt['Jaar'].unique()

array(['2013', '2014', '2015', '2016', '2017'], dtype=object)

In [20]:
dscMelt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847075 entries, 0 to 847074
Data columns (total 13 columns):
nr                    847075 non-null int64
PC4_LEERL             847075 non-null int64
KWALIFICATIENIVEAU    847075 non-null int64
KWALIFICATIE CODE     847075 non-null int64
KWALIFICATIE NAAM     847075 non-null object
MBO SECTOR            847075 non-null object
DOMEIN                847075 non-null object
SECTORUNIT SBB        847075 non-null object
GESLACHT              847075 non-null object
Locatie               847075 non-null object
Categorie             847075 non-null object
Jaar                  847075 non-null object
Aantal                847075 non-null int64
dtypes: int64(5), object(8)
memory usage: 84.0+ MB


# 2. Voorbeeld in Bokeh:

Poging een voorbeeld te maken in Bokeh

In [49]:
# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.
df = dscMelt.copy()

# defineer variabelen voor grafiek
df_totaalSBB = df.groupby(['SECTORUNIT SBB', 'Jaar'], as_index=False).sum()[['SECTORUNIT SBB', 'Jaar', 'Aantal']]
df_totaalSBB.head()

# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.
dx = dscMelt.copy()

# defineer variabelen voor grafiek
dx_totaalSBB = dx.groupby(['SECTORUNIT SBB', 'Jaar']).sum()[['Aantal']]
dx_totaalSBB.head()

Aantal
SECTORUNIT SBB Jaar        
Bovensectoraal 2013     369
               2014     517
               2015     433
               2016     306
               2017     562

In [22]:
df_totaalSBB.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 0 to 44
Data columns (total 3 columns):
SECTORUNIT SBB    45 non-null object
Jaar              45 non-null object
Aantal            45 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.4+ KB


In [23]:
#Defineer brondata & kolommen
source = ColumnDataSource(df_totaalSBB)
original_source = ColumnDataSource(df_totaalSBB)
columns = [
   TableColumn(field="SECTORUNIT SBB", title="Sector"),
   TableColumn(field="Jaar", title="Jaar"),
   TableColumn(field="Aantal", title="Aantal"),
]

In [50]:
# Voorbeeld datatable output voor alle datatabellen.
output_notebook()

#Defineer brondata & kolommen
source = ColumnDataSource(df_totaalSBB)
original_source = ColumnDataSource(df_totaalSBB)
columns = [
   TableColumn(field="SECTORUNIT SBB", title="Sector"),
   TableColumn(field="Jaar", title="Jaar"),
   TableColumn(field="Aantal", title="Aantal"),
]

#maak data tabel
data_table = DataTable(source=source, columns=columns)

# callback code to be used by all the filter widgets
combined_callback_code = """
var data = source.data;
var original_data = original_source.data;
var sector = sector_select_obj.value;
console.log("sector: " + sector);
var jaar = jaar_select_obj.value;
console.log("jaar: " + jaar);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['SECTORUNIT SBB'].length; ++i) {
        if ((sector === "ALL" || original_data['SECTORUNIT SBB'][i] === sector) &&
            (jaar === "ALL" || original_data['Jaar'][i] === jaar)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.change.emit();;
source.change.emit();
"""

# define the filter widgets, without callbacks for now
sector_list = ['ALL'] + df_totaalSBB['SECTORUNIT SBB'].unique().tolist()
sector_select = Select(title="Sector:", value=sector_list[0], options=sector_list)
jaar_list = ['ALL'] + df_totaalSBB['Jaar'].unique().tolist()
jaar_select = Select(title="Jaar:", value=jaar_list[0], options=jaar_list)

# now define the callback objects now that the filter widgets exist
generic_callback = CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              sector_select_obj=sector_select, 
              jaar_select_obj=jaar_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
sector_select.js_on_change('value', generic_callback)
jaar_select.js_on_change('value', generic_callback)


#layout = row(p, jaren)
p = column(sector_select,jaar_select,data_table)

#doc.title = "Sectorunit SBB chart test"
#doc.add_root(p)
show(p)


Loading BokehJS ...

In [51]:
# Datatabel + grafiek, nog in 'kolom' layout - werkend!!! 

#def make_document(doc):
#output_notebook()

#Defineer brondata & kolommen
source = ColumnDataSource(df_totaalSBB)
original_source = ColumnDataSource(df_totaalSBB)
columns = [
   TableColumn(field="SECTORUNIT SBB", title="Sector"),
   TableColumn(field="Jaar", title="Jaar"),
   TableColumn(field="Aantal", title="Aantal"),
]

#maak data tabel
data_table = DataTable(source=source, columns=columns)

# callback code to be used by all the filter widgets
combined_callback_code = """
var data = source.data;
var original_data = original_source.data;
var sector = sector_select_obj.value;
console.log("sector: " + sector);
var jaar = jaar_select_obj.value;
console.log("jaar: " + jaar);
for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['SECTORUNIT SBB'].length; ++i) {
        if ((sector === "ALL" || original_data['SECTORUNIT SBB'][i] === sector) &&
            (jaar === "ALL" || original_data['Jaar'][i] === jaar)) {
            data[key].push(original_data[key][i]);
        }
    }
}
target_obj.change.emit();;
source.change.emit();
"""

# define the filter widgets, without callbacks for now
sector_list = ['ALL'] + df_totaalSBB['SECTORUNIT SBB'].unique().tolist()
sector_select = Select(title="Sector:", value=sector_list[0], options=sector_list)
jaar_list = ['ALL'] + df_totaalSBB['Jaar'].unique().tolist()
jaar_select = Select(title="Jaar:", value=jaar_list[0], options=jaar_list)

# now define the callback objects now that the filter widgets exist
generic_callback = CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              sector_select_obj=sector_select, 
              jaar_select_obj=jaar_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
sector_select.js_on_change('value', generic_callback)
jaar_select.js_on_change('value', generic_callback)


# add chart 

kolommen = list(df_totaalSBB['Jaar'].unique())
sectoren = list(df_totaalSBB['SECTORUNIT SBB'].unique())

# voorbereiden dataset
df_copyForStack =  dsc.copy()
df_copyForStackGrouped = df_copyForStack.groupby(['SECTORUNIT SBB']).sum()[['2013', '2014', '2015', '2016', '2017']]

# Groepeer voor volgorde hoog-laag
df_copyForStackGrouped.sort_values(by=['2013', '2014', '2015', '2016', '2017'], ascending=False, inplace=True)

# Zet om naar dictionary voor verwerking door grafiek
df_stackedToDict = df_copyForStackGrouped.to_dict('list')
df_stackedToDict['sectoren'] = sectoren

# Maak er een CDS van
source2 = ColumnDataSource(df_stackedToDict)

#Teken grafiek
p1 = figure(x_range=sectoren, plot_height=800, title="Totalen per SBB sector per jaar")
p1.vbar_stack(kolommen, x='sectoren', width=0.8, source=source2, legend=kolommen, color=['#084594', '#2171b5', '#4292c6', '#6baed6', '#9ecae1'])

p1.width=900
p1.y_range.start = 0
p1.x_range.range_padding = 0.1
p1.xaxis.major_label_orientation = 'vertical'
p1.xgrid.grid_line_color = None
p1.left[0].formatter.use_scientific = False


#layout = row(p, jaren)
p = column(sector_select,jaar_select,data_table, p1)

#doc.title = "Sectorunit SBB chart test"
#doc.add_root(p)
show(p)

In [ ]:
## -- test --

#source2 = ColumnDataSource(dx_totaalSBB)
kolommen = list(df_totaalSBB['Jaar'].unique())
sectoren = list(df_totaalSBB['SECTORUNIT SBB'].unique())
df_copyForStack =  dsc.copy()
df_copyForStackGrouped = df_copyForStack.groupby(['SECTORUNIT SBB']).sum()[['2013', '2014', '2015', '2016', '2017']]
df_stackedToDict = df_copyForStackGrouped.to_dict('list')
df_stackedToDict['sectoren'] = sectoren


#Teken grafiek
source2 = ColumnDataSource(df_stackedToDict)

p1 = figure(x_range=sectoren, plot_height=800, title="Totalen per SBB sector per jaar")

p1.vbar_stack(kolommen, x='sectoren', width=0.8, source=data2, legend=kolommen, color=['#084594', '#2171b5', '#4292c6', '#6baed6', '#9ecae1'])

p1.width=900
p1.y_range.start = 0
p1.x_range.range_padding = 0.1
p1.xaxis.major_label_orientation = 'vertical'
p1.xgrid.grid_line_color = None
p1.left[0].formatter.use_scientific = False

#layout = row(p, jaren)
#doc.title = "Sectorunit SBB chart test"
#doc.add_root(p)
show(p1)

In [ ]:
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler

apps = {'/': Application(FunctionHandler(make_document))}

server = Server(apps, port=5006, allow_websocket_origin=['localhost:5006','nm-interactive.net:5006'])
server.start()

In [ ]:
server.stop()


In [ ]:
## -- test --

# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.
dx = dscMelt.copy()

# defineer variabelen voor grafiek
dx_totaalSBB = dx.groupby(['SECTORUNIT SBB', 'Jaar']).sum()[['Aantal']]
dx_totaalSBB.head()
dx_totaalSBB.loc['Bovensectoraal']

# 2. Visualisatie op 'Sectorunit SBB'
De aanvliegroute is om per visualisatie een apart dataframe te maken. Sorteren op attribuurtwaarde lijkt me handig.

### 1a. Dataframe verkennen: Sorteren op unieke attribuutwaarde
Normaal gesproken staan alle data in willekeurige volgorde. Je zou kunnen sorteren attribuurtwaarde, bijvoorbeeld binnen de kolom 'SECTORUNIT SBB'.<br>
<font color=grey>
Syntax: df.sort_values(by=['col1']) , of<br>
df.sort_values(by=['col1', 'col2']) <br>
Wil je eea omgekeerd selecteren? Dan df.sort_values(by='col1', ascending=False)</font>
<font color=grey>Bron:https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html</font> <br>

N.B. Simon en Danny zijn nog bezig met 'Locatie' en 'Categorie'. Dit zijn nu arrays [] binnen de Dataframe. Ze zoeken nog naar een manier om dit om te typeren naar normale datavelden, zodat we ook hier aparte Dataframes (locatie, categorie) van kunnen maken die gevisualiseerd kunnen worden.



In [ ]:
df = datasource
df.sort_values(by='SECTORUNIT SBB')

### 1b. SNELLER: Welke unieke attribuutwaarden in een kolom
Het is sneller om de unieke attribuutwaarden uit te vragen. Dan is het later makkelijker om te selecteren / slicen / visualiseren op een bepaalde attribuutwaarde in een kolom.

Bron:https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/ 

In [ ]:
df['SECTORUNIT SBB'].unique()

# 2. Totalen per unieke attribuutwaarde
Om de unieke attribuutwaarden te visualiseren, hebben we de totalen nodig. Schijnbaar kan dat ook makkelijk, na lang zoeken, gvd. <br>

<font color=grey>Syntax: df.groupby(['kolomnaam' waarvan je totalen wil]).sum()[['kolom aant1','kolom aant1', 'enzovoorts']]</font> <br>
Bron: https://stackoverflow.com/questions/35307732/how-to-sum-in-pandas-by-unique-index-in-several-columns

In [ ]:
df_totaalSBB = df.groupby(['SECTORUNIT SBB']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_totaalSBB.info()

In [ ]:
df_totaalSBB.head(10)

# 3. Visualiseren totalen 'Sectorunit SBB'

In [ ]:
%config IPCompleter.greedy = True  
%matplotlib inline
df_totaalSBB.plot.bar()

Bron voor onderstaande: https://hub.mybinder.org/user/bokeh-bokeh-notebooks-yecjmjgh/notebooks/tutorial/07%20-%20Bar%20and%20Categorical%20Data%20Plots.ipynb

# 4. Visualiseer hoe vaak een bepaald type mbo niveau 4 gediplomeerden voorkomt in het wervingsgebied van de Windesheim locatie Zwolle in absolute aantallen

In [ ]:
#dataframe om alle gediplomeerden in regio Zwolle te selecteren
df_locatie_zwolle = df.loc[df['Locatie'] == 'Zwolle']
df_aantal_zwolle = df_locatie_zwolle.groupby(['Locatie']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]

In [ ]:
df_aantal_zwolle.info()

In [ ]:
df_aantal_zwolle.plot.bar(figsize=(8,5.0),title='Aantal MBO Afgestudeerden Regio Zwolle')

In [ ]:
df_locatie_zwolle.info()

# 5. Visualiseer hoe vaak een bepaald type mbo niveau 4 gediplomeerden voorkomt in het wervingsgebied van de Windesheim locatie Flevoland in absolute aantallen

In [ ]:
#dataframe om alle gediplomeerden in regio Flevoland te selecteren
df_locatie_flevoland = df.loc[df['Locatie'] == 'Flevoland']
df_aantal_flevoland = df_locatie_flevoland.groupby(['Locatie']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]

In [ ]:
df_aantal_flevoland

In [ ]:
df_aantal_flevoland.plot.bar(figsize=(8,5.0),title='Aantal MBO Afgestudeerden Regio Flevoland')

# JH - 1. Visualiseer waar de gediplomeerde mbo-4 woonachtig zijn naar mbo-opleiding in absolute aantallen

In [ ]:
koppeltabel = pd.read_sql_query("SELECT * FROM koppeltabel",engine)

In [ ]:
renamed = datasource.rename(columns={'PC4_LEERL': 'PC4'})
output = pd.merge(renamed, koppeltabel, on='PC4', how='inner')
output.head(10)

# JH - 1A. Visualisatie per Provincie

In [ ]:
#dataframe om alle gediplomeerden per provincie te selecteren
df_totaalprovincie = output.groupby(['Provincie']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_totaalprovincie

In [ ]:
df_totaalprovincie.plot.bar(figsize=(8,5.0),title='Aantal MBO 4 Afgestudeerden per Provincie')

# JH - 1B. Visualisatie per Categorie

In [ ]:
#dataframe om alle gediplomeerden per categorie te selecteren
df_totaalcategorie = output.groupby(['Categorie']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_totaalcategorie

In [ ]:
df_totaalcategorie.plot.bar(figsize=(8,5.0),title='Aantal MBO 4 Afgestudeerden per Categorie')

# JH - 1C. Visualisatie "Thuismarkt"

In [ ]:
#dataframe om alle gediplomeerden in Categorie "Thuismarkt" te selecteren
df_thuismarkt = output.loc[output['Categorie'] == 'Thuismarkt']
df_gt = df_thuismarkt.groupby(['Gemeente']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_gt

In [ ]:
df_gt.plot.bar(figsize=(20,8.0),title='Aantal MBO 4 Afgestudeerden - Thuismarkt')

# JH - 1D. Visualisatie "Aangrenzend Primair"

In [ ]:
#dataframe om alle gediplomeerden in Categorie "Aangrenzend Primair" te selecteren
df_ap = output.loc[output['Categorie'] == 'Aangrenzend Primair']
df_gap = df_ap.groupby(['Gemeente']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_gap

In [ ]:
df_gap.plot.bar(figsize=(20,8.0),title='Aantal MBO 4 Afgestudeerden - Aangrenzend Primair')

# JH - 1E. Visualisatie "Aangrenzend Secundair"

In [ ]:
#dataframe om alle gediplomeerden in Categorie "Aangrenzend Secundair" te selecteren
df_as = output.loc[output['Categorie'] == 'Aangrenzend Secundair']
df_gas = df_as.groupby(['Gemeente']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_gas

In [ ]:
df_gas.plot.bar(figsize=(20,8.0),title='Aantal MBO 4 Afgestudeerden - Aangrenzend Secundair')

# JH - 1F. Visualisatie "Meerdere"

In [ ]:
#dataframe om alle gediplomeerden in Categorie "Meerdere" te selecteren
df_me = output.loc[output['Categorie'] == 'Meerdere']
df_gme = df_me.groupby(['Gemeente']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_me

In [ ]:
df_gme.plot.bar(figsize=(20,8.0),title='Aantal MBO 4 Afgestudeerden - Aangrenzend Secundair')

# 2. Visualisatie "MBO Sector" in absolute aantallen

In [ ]:
df_sector = df.groupby(['MBO SECTOR']).sum()[['2013   AANT','2014   AANT','2015   AANT','2016   AANT','2017   AANT']]
df_sector

In [ ]:
df_sector.plot.bar(figsize=(20,8.0),title='Aantal MBO 4 Afgestudeerden - per MBO Sector')

In [ ]:
#output_notebook()

#Kolommen zijn de kolommen uit de df_totaalSBB dataframe. Dit gaat dan om 2013 AANT etc. Dit is voor de eerste groepering. 
kolommen = list(df_totaalSBB)

#Sectoren haalt alle unieke waardes uit het dataframe voor de kolom SECTORUNIT SBB.
sectoren = list(df['SECTORUNIT SBB'].unique())

#Voeg per kolom alle sectoren toe. Dit wordt dan de data voor de x-as. 
#Kolom is bijvoorbeeld 2013 AANT en hier worden vervolgens alle sectoren aan toegevoegd.
x = [ (kolom, sector) for kolom in kolommen for sector in sectoren ]

#Data voor de y-as
#Data doorlopen om de totalen op de juiste volgorde in een array te plaatsen.
#Zelfde volgorde zoals hierboven staat voor de gegevens voor de x-as

counts = []

for kolom in kolommen:
    for sector in sectoren:
        counts.append(df_totaalSBB.loc[sector][kolom])
        
#Teken grafiek
source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), plot_height=400, title="Totalen per SBB sector per jaar")

p.vbar(x='x', top='counts', width=0.8, source=source)

p.width=900
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [ ]:
#Kolommen zijn de kolommen uit de df_totaalSBB dataframe. Dit gaat dan om 2013 AANT etc. Dit is voor de eerste groepering. 
kolommen_zwolle = list(df_aantal_zwolle)

#Sectoren haalt alle unieke waardes uit het dataframe voor de kolom SECTORUNIT SBB.
locaties_zwolle = list(df_locatie_zwolle['Locatie'].unique())

#Voeg per kolom alle sectoren toe. Dit wordt dan de data voor de x-as. 
#Kolom is bijvoorbeeld 2013 AANT en hier worden vervolgens alle sectoren aan toegevoegd.
x = [ (kolom, locatie) for kolom in kolommen_zwolle for locatie in locaties_zwolle ]

#Data voor de y-as
#Data doorlopen om de totalen op de juiste volgorde in een array te plaatsen.
#Zelfde volgorde zoals hierboven staat voor de gegevens voor de x-as

counts = []

for kolom in kolommen_zwolle:
    for locatie in locaties_zwolle:
        counts.append(df_aantal_zwolle.loc[locatie][kolom])

#Teken grafiek
source = ColumnDataSource(data=dict(x=x, counts=counts))

az = figure(x_range=FactorRange(*x), plot_height=400, title="Aantal MBO Afgestudeerden Regio Zwolle")

az.vbar(x='x', top='counts', width=0.8, source=source)

az.width=900
az.y_range.start = 0
az.x_range.range_padding = 0.1
az.xaxis.major_label_orientation = 1
az.xgrid.grid_line_color = None


#update functie om nieuwe data(selecties) weer te geven, doet nu niets...
#def update():

#callback om updates elke 100ms op te halen
#doc.add_periodic_callback(update, 100)

show(az)

In [ ]:
#from bokeh.palettes import Spectral6

#Kolommen zijn de kolommen uit de df_totaalSBB dataframe. Dit gaat dan om 2013 AANT etc. Dit is voor de eerste groepering. 
#kolommen = list(df_totaalSBB)

#Sectoren haalt alle unieke waardes uit het dataframe voor de kolom SECTORUNIT SBB.
#sectoren = df['SECTORUNIT SBB'].unique()
#sectorenMultiplied = []

#Voeg per kolom alle sectoren toe. Dit wordt dan de data voor de x-as. 
#Kolom is bijvoorbeeld 2013 AANT en hier worden vervolgens alle sectoren aan toegevoegd.
#x = [ (sector, kolom) for sector in sectoren for kolom in kolommen ]

#Data voor de y-as
#Data doorlopen om de totalen op de juiste volgorde in een array te plaatsen.
#Zelfde volgorde zoals hierboven staat voor de gegevens voor de x-as

#counts = []

#for sector in sectoren:
#    for kolom in kolommen:
#        counts.append(df_totaalSBB.loc[sector][kolom])

#cyl_cmap = factor_cmap('cyl', palette=Spectral5, factors=sorted(df.cyl.unique())
#Teken grafiek
#source = ColumnDataSource(data=dict(x=x, counts=counts, sectoren=sectorenMultiplied, color=Spectral6))
#p = figure(x_range=FactorRange(*x), plot_height=400, title="Totalen per SBB sector per jaar")
#p.vbar(x='x', top='counts', width=0.8, legend='sectoren', color='color', source=source)

#p.width=1000
#p.y_range.start = 0
#p.x_range.range_padding = 0.1
#p.xaxis.major_label_orientation = 'vertical'
#p.xgrid.grid_line_color = None

#p.legend.orientation = "horizontal"
#p.legend.location = "top_center"

#p.xaxis[0].group_text_font_size = '0pt'
#show(p)

In [ ]:
# voorbeeld data, laad in om werking te zien! Wel even de volgende paragraaf skippen...

import random

def make_document(doc):
    source = ColumnDataSource({'x': [], 'y': [], 'color': []})

    def update():
        new = {'x': [random.random()],
               'y': [random.random()],
               'color': [random.choice(['red', 'blue', 'green'])]}
        source.stream(new)

    doc.add_periodic_callback(update, 100)

    fig = figure(title='Streaming Circle Plot!', sizing_mode='scale_width',
                 x_range=[0, 1], y_range=[0, 1])
    fig.circle(source=source, x='x', y='y', color='color', size=10)

    doc.title = "Now with live updating!"
    doc.add_root(fig)

In [ ]:
# maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
# weergegeven met meerdere opties.

def make_document(doc):
    #Kolommen zijn de kolommen uit de df_totaalSBB dataframe. Dit gaat dan om 2013 AANT etc. Dit is voor de eerste groepering. 
    kolommen = list(df_totaalSBB)

    #Sectoren haalt alle unieke waardes uit het dataframe voor de kolom SECTORUNIT SBB.
    sectoren = list(df['SECTORUNIT SBB'].unique())

    #Voeg per kolom alle sectoren toe. Dit wordt dan de data voor de x-as. 
    #Kolom is bijvoorbeeld 2013 AANT en hier worden vervolgens alle sectoren aan toegevoegd.
    x = [ (kolom, sector) for kolom in kolommen for sector in sectoren ]

    #Data voor de y-as
    #Data doorlopen om de totalen op de juiste volgorde in een array te plaatsen.
    #Zelfde volgorde zoals hierboven staat voor de gegevens voor de x-as

    counts = []

    for kolom in kolommen:
        for sector in sectoren:
            counts.append(df_totaalSBB.loc[sector][kolom])

    #Teken grafiek
    source = ColumnDataSource(data=dict(x=x, counts=counts))

    p = figure(x_range=FactorRange(*x), plot_height=400, title="Totalen per SBB sector per jaar")

    p.vbar(x='x', top='counts', width=0.8, source=source)

    p.width=900
    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None


    #update functie om nieuwe data(selecties) weer te geven
    def updateSBB(attr, old, new):
        kolommen = [jaren.labels[i] for i in jaren.active]
        x = [ (kolom, sector) for kolom in kolommen for sector in sectoren ]

        counts = []

        for kolom in kolommen:
            for sector in sectoren:
                counts.append(df_totaalSBB.loc[sector][kolom])

        newSource = ColumnDataSource(data=dict(x=x, counts=counts))
        source.data.update(newSource.data)
    
    
    jaren = CheckboxGroup(labels=kolommen, active = [0, 1, 2, 3 ,4])
    jaren.on_change('active', updateSBB)

    layout = row(p, jaren)

    #callback om updates elke 100ms op te halen
    #doc.add_periodic_callback(update, 100)

    doc.title = "Sectorunit SBB chart..."
    doc.add_root(layout)

    # maak document, met daarin grafiek. Hier zullen (uiteindelijk) meerdere grafieken moeten worden 
    # weergegeven met meerdere opties.

    #def aantal_zwolle(doc):

        #Kolommen zijn de kolommen uit de df_totaalSBB dataframe. Dit gaat dan om 2013 AANT etc. Dit is voor de eerste groepering. 
        #kolommen_zwolle = list(df_aantal_zwolle)

        #Sectoren haalt alle unieke waardes uit het dataframe voor de kolom SECTORUNIT SBB.
        #locaties_zwolle = list(df_aantal_zwolle['Locatie'].unique())

        #Voeg per kolom alle sectoren toe. Dit wordt dan de data voor de x-as. 
        #Kolom is bijvoorbeeld 2013 AANT en hier worden vervolgens alle sectoren aan toegevoegd.
        #x = [ (kolom, locatie) for kolom in kolommen_zwolle for locatie in locaties_zwolle ]

        #Data voor de y-as
        #Data doorlopen om de totalen op de juiste volgorde in een array te plaatsen.
        #Zelfde volgorde zoals hierboven staat voor de gegevens voor de x-as

        #counts = []

        #for kolom in kolommen_zwolle:
        #   for locatie in locaties_zwolle:
        #        counts.append(df_aantal_zwolle.loc[sector][kolom])

        #Teken grafiek
        #source = ColumnDataSource(data=dict(x=x, counts=counts))

        #az = figure(x_range=FactorRange(*x), plot_height=400, title="Aantal MBO Afgestudeerden Regio Zwolle")

        #az.vbar(x='x', top='counts', width=0.8, source=source)

        #az.width=900
        #az.y_range.start = 0
        #az.x_range.range_padding = 0.1
        #az.xaxis.major_label_orientation = 1
        #az.xgrid.grid_line_color = None
   

        #update functie om nieuwe data(selecties) weer te geven, doet nu niets...
        #def update():

        #callback om updates elke 100ms op te halen
        #doc.add_periodic_callback(update, 100)
        
        
        #doc.title = "Tekst..."
        #doc.add_root(az)

In [ ]:
# starten van de webserver :)
apps = {'/': Application(FunctionHandler(make_document))}

server = Server(apps, port=5006, allow_websocket_origin=['localhost:5006','nm-interactive.net:5006'])
server.start()

In [ ]:
# stoppen van de server :)
server.stop()

In [ ]:
output_notebook()

#Kolommen zijn de kolommen uit de df_totaalSBB dataframe. Dit gaat dan om 2013 AANT etc. Dit is voor de eerste groepering. 
kolommen = list(df_totaalSBB)

#Sectoren haalt alle unieke waardes uit het dataframe voor de kolom SECTORUNIT SBB.
sectoren = df['SECTORUNIT SBB'].unique()

#Voeg per kolom alle sectoren toe. Dit wordt dan de data voor de x-as. 
#Kolom is bijvoorbeeld 2013 AANT en hier worden vervolgens alle sectoren aan toegevoegd.
x = [ (kolom, sector) for kolom in kolommen for sector in sectoren ]

#Data voor de y-as
#Data doorlopen om de totalen op de juiste volgorde in een array te plaatsen.
#Zelfde volgorde zoals hierboven staat voor de gegevens voor de x-as

counts = []

for kolom in kolommen:
    for sector in sectoren:
        counts.append(df_totaalSBB.loc[sector][kolom])
        

source = ColumnDataSource(data=dict(x=x, counts=counts))

#Teken grafiek
p = figure(x_range=FactorRange(*x), plot_height=400, title="Totalen per SBB sector per jaar")

p.vbar(x='x', top='counts', width=0.8, source=source)

p.width=900
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [ ]:
def updateSBB(attr, old, new):
    kolommen = [jaren.labels[i] for i in jaren.active]
    x = [ (kolom, sector) for kolom in kolommen for sector in sectoren ]
   
    counts = []
    
    for kolom in kolommen:
        for sector in sectoren:
            counts.append(df_totaalSBB.loc[sector][kolom])
    
    newSource = ColumnDataSource(data=dict(x=x, counts=counts))
    source.data.update(newSource.data)

In [ ]:
output_file("slider.html")
jaren = CheckboxGroup(labels=kolommen, active = [0, 1, 2, 3 ,4])
jaren.on_change('active', updateSBB)

p1 = figure(plot_width=300, plot_height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="red", alpha=0.5)

testcheckbox = CheckboxGroup(labels=list("Test"), active = [0, 1])
                             
layout = row(p, jaren)
tab1 = Panel(child=layout, title="Aantal per SBB Sectorunit")

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="red", alpha=0.5)

l2 = row(az)
tab2 = Panel(child=l2, title="Aantal MBO 4 afgestudeerden Zwolle")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)

In [ ]:
def test():
    print("test")

In [ ]:
test()